# Packages Installs and environment preparation

In [7]:
# !7z x data/20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS.7z
# !7z x data/20200427_UOWM_IEC104_Dataset_c_se_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_sc_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS.7z
# !7z x data/20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1.7z
# !7z x data/20200608_UOWM_IEC104_Dataset_mitm_drop.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1.7z

# Imports

In [8]:
from scapy.all import *
import pandas as pd
# from catboost import CatBoostClassifier
# import shap
import seaborn as sns
# import feature_engine
# import feature_engine.encoding
import numpy as np
# from catboost import Pool
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
# from google.colab import drive
# drive.mount('/content/drive')

## Code

In [9]:
def get_attack_directory_attack(path):
  return path.split('Dataset_')[1]
attacks_directories= [
                      '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1',
                      '20200427_UOWM_IEC104_Dataset_c_se_na_1',
                      '20200428_UOWM_IEC104_Dataset_c_sc_na_1',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS',
                      '20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS',
                      '20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS',
                                            # '20200608_UOWM_IEC104_Dataset_mitm_drop'
                                            ]
devices_to_read = ['qtester','attacker1','attacker2','attacker3']

# if '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
#   not
get_attack_directory_attack(attacks_directories[0])

'm_sp_na_1_DoS'

In [10]:
a={}
for attack_directory in attacks_directories:
  for device_to_read in devices_to_read:
    pcap_file = f"data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    packets=rdpcap(pcap_file,count=200)
    for packet in packets:
      a[len(packet[TCP].payload)] = 1
MAX_IEC104_PAYLOAD_BYTES = max(a)

In [11]:
MAX_IEC104_PAYLOAD_BYTES

88

In [12]:
from pathlib import Path
all_packets = []
for attack_directory in attacks_directories:
#   dfs=[]
  print(attack_directory)
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"./data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    # df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
    packets=rdpcap(pcap_file)
    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'
    all_packets.append([packets,label,get_attack_directory_attack(attack_directory)])
    # df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    # df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    # dfs.append(df)
#   df = pd.concat(dfs)
#   df.to_parquet(f"./data/iec104v2_15.parquet")

20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS
20200426_UOWM_IEC104_Dataset_c_ci_na_1
20200427_UOWM_IEC104_Dataset_c_se_na_1
20200428_UOWM_IEC104_Dataset_c_sc_na_1
20200605_UOWM_IEC104_Dataset_c_rd_na_1
20200606_UOWM_IEC104_Dataset_c_rp_na_1
20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS
20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS
20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS
20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS
20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS


In [13]:
def tcp_packet_to_binary(tcp_packet):
    """
    Converts a TCP packet into a binary sequence.
    
    Args:
        tcp_packet (TCP): The TCP packet to be converted.
    
    Returns:
        numpy.ndarray: The binary sequence representing the TCP packet.
    """
    # Extract fields from the TCP packet
    sport = tcp_packet.sport
    dport = tcp_packet.dport
    seq = tcp_packet.seq
    ack = tcp_packet.ack
    dataofs_reserved = (tcp_packet.dataofs << 4) | tcp_packet.reserved
    flags = tcp_packet.flags
    window = tcp_packet.window
    chksum = tcp_packet.chksum
    urgptr = tcp_packet.urgptr

    # Convert each field to binary and concatenate to form the binary sequence
    binary_sequence = (
        format(sport, '016b') +
        format(dport, '016b') +
        format(seq, '032b') +
        format(ack, '032b') +
        format(dataofs_reserved, '08b') +
        format(flags, '08b') +
        format(window, '016b') +
        format(chksum, '016b') +
        format(urgptr, '016b')
    )
    
    # Convert the binary string to a NumPy array
    binary_array = np.array([int(bit) for bit in binary_sequence], dtype=int)
    
    return binary_array


def ipv4_packet_to_binary(ip_packet):
    """
    Converts an IPv4 packet into a binary sequence.
    
    Args:
        ip_packet (IP): The IPv4 packet to be converted.
    
    Returns:
        numpy.ndarray: The binary sequence representing the IPv4 packet.
    """
    # First, ensure that the packet is an IP packet
    if not (IP in ip_packet):
        raise ValueError("The provided packet is not an IP packet.")
    
    # Extract source and destination IP addresses and split them into octets
    try:
        src_ip = ip_packet.src.split('.')
        dst_ip = ip_packet.dst.split('.')
    except AttributeError:
        raise ValueError("The packet does not contain standard IP fields.")

    # Convert each field to binary and concatenate to form the binary sequence
    binary_sequence = (
        format((ip_packet.version << 4) | ip_packet.ihl, '08b') +
        format(ip_packet.tos, '08b') +
        format(ip_packet.len, '016b') +
        format(ip_packet.id, '016b') +
        format((ip_packet.flags << 13) | ip_packet.frag, '016b') +
        format(ip_packet.ttl, '08b') +
        format(ip_packet.proto, '08b') +
        format(ip_packet.chksum, '016b') +
        ''.join([format(int(octet), '08b') for octet in src_ip]) +
        ''.join([format(int(octet), '08b') for octet in dst_ip])
    )

    # Convert the binary string to a NumPy array
    binary_array = np.array([int(bit) for bit in binary_sequence], dtype=int)
    
    return binary_array

In [14]:
def session_extractor(packet,attack):
    return str(sorted([attack, "TCP", packet["IP"].src, packet["TCP"].sport,
                                    packet["IP"].dst, packet["TCP"].dport], key=str))

In [15]:
# ipv4_packet_to_binary(all_packets[0][0][0])

In [25]:
dfs= []
for packets, label, attack_directory_attack in all_packets:
    session_dict = packets.sessions(lambda x: session_extractor(x,attack_directory_attack))
    for k,v in session_dict.items():
        print(k,v)
        break
    # dfs.append(pd.concat({flow_key: build_dataframe(flow) for flow_key, flow in session_dict.items()}))
    break

['192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS'] <PacketList: TCP:26601 UDP:0 ICMP:0 Other:0>


In [33]:
dfs= []
packets, label, attack_directory_attack = all_packets[2]
session_dict = packets.sessions(lambda x: session_extractor(x,attack_directory_attack))
for k,v in session_dict.items():
    print(k,v)
    for i in v:
        print(i.flags)
    break
    # break
# dfs.append(pd.concat({flow_key: build_dataframe(flow) for flow_key, flow in session_dict.items()}))


['192.168.1.21', '192.168.1.28', 2404, 33491, 'TCP', 'c_ci_na_1'] <PacketList: TCP:9 UDP:0 ICMP:0 Other:0>
DF
DF
DF
DF
DF
DF
DF
DF
DF


In [17]:
# dfs.append(pd.concat([build_dataframe(flow) for flow_key, flow in session_dict.items()]))


In [18]:
# dfs= []
# for packets, label, attack_directory_attack in all_packets:
#     session_dict = packets.sessions(lambda x: session_extractor(x,attack_directory_attack))
#     dfs.append(pd.concat({flow_key: build_dataframe(flow) for flow_key, flow in session_dict.items()}))
#     break

In [19]:
dfs= pd

In [20]:
pcap_file = f"./data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
packets=rdpcap(pcap_file)

In [21]:
# # LAST_PACKET_TIME = defaultdict(lambda: 0)  
# # TIMEOUT = 0.00000001
# def session_extractor(packet,attack):
#     return str(sorted([attack, "TCP", packet["IP"].src, packet["TCP"].sport,
#                                     packet["IP"].dst, packet["TCP"].dport], key=str))
# session_dict =packets.sessions(session_extractor)
# # len(session_dict)

In [22]:
len(session_dict)

NameError: name 'session_dict' is not defined

In [ ]:
session_dict["['192.168.1.19', '192.168.1.29', 2404, 42239, 'TCP']_1588157787"].show()

In [ ]:
from pathlib import Path
for attack_directory in attacks_directories:
  dfs=[]
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"./data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    # df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
    packets=rdpcap(pcap_file)
    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'

    df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    dfs.append(df)
  df = pd.concat(dfs)
  df.to_parquet(f"./data/iec104v2_15.parquet")

ValueError: All arrays must be of the same length

In [ ]:

from pathlib import Path

for attack_directory in attacks_directories:
  dfs=[]
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"/content/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    # df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
    packets=rdpcap(pcap_file)
    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'

    df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    dfs.append(df)
  df = pd.concat(dfs)
  df.to_parquet(f"/content/drive/My Drive/IEC104/{attack_directory}.parquet")

In [ ]:
!wc -l /content/20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS_attacker2_iec104_only.pcap.npt
!head /content/20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS_attacker2_iec104_only.pcap.npt

In [ ]:
# datasets=[]
# store=pd.HDFStore('iec104_all.h5')
# #     for df in dfs:
# #
# #     #del dfs
# #     df=store.select('iec104')
# #     store.close()
# #     os.remove('iec104_all.h5')
# #     return df
# for attack_directory in attacks_directories:
#   for device_to_read in devices_to_read:
#     if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
#       continue
#     pcap_file = f"/content/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
#     # print(device_to_read,attack_directory)
#     df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
#     if device_to_read != 'qtester':
#       df['label'] = get_attack_directory_attack(attack_directory)
#     else:
#       df['label'] = 'normal'
#     store.append('iec104',df,data_columns=df.columns)
#     # df['flow']=df['flow']
#     # print(df.head())
#     # datasets.append(df)
# store.close()

In [ ]:
# def hdf_save(dfs):
#     store=pd.HDFStore('iec104_all.h5')
#     for df in dfs:
#         store.append('iec104',df,data_columns=df.columns)
#     #del dfs
#     df=store.select('iec104')
#     store.close()
#     os.remove('iec104_all.h5')
#     return df
# hdf_save(datasets)

In [ ]:
# datasets=[]
from pathlib import Path

for attack_directory in attacks_directories:
  dfs=[]
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"/content/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)

    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'

    df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    dfs.append(df)
  df = pd.concat(dfs)
  df.to_parquet(f"/content/drive/My Drive/IEC104/{attack_directory}.parquet")

In [ ]:
# datasets=[]
# for attack_directory in attacks_directories:
#   for device_to_read in devices_to_read:
#     if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
#       continue
#     pcap_file = f"/content/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
#     # print(device_to_read,attack_directory)
#     df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
#     if device_to_read != 'qtester':
#       df['label'] = get_attack_directory_attack(attack_directory)
#     else:
#       df['label'] = 'normal'
#     # df['flow']=df['flow']
#     # print(df.head())
#     datasets.append(df)

In [ ]:
# # df = pd.concat(datasets)
# # df.head(3)
# df = pd.DataFrame()  # Initialize an empty DataFrame to store the concatenated data

# for dataset in datasets:
#     df = pd.concat([df, dataset])
#     del dataset  # Delete the original dataset to free up memory

In [ ]:
# df.shape

In [ ]:
# df.to_parquet("/content/drive/My Drive/IEC104/iec104.parquet")